## Testing Code

In [74]:
#  Practice:
example = {
    "id": "22a4636f-8179-4357-8e87-d1743ece1f81",
    "snapshot": "chrome",
    "instruction": "Please help me convert the file \"Meeting-Agenda.docx\" to a pdf file and upload to the folder 'meetings' in my google drive.",
    "source": "https://marketplace.uipath.com/listings/merge-pdfs-from-gmail-email-attachments-and-upload-to-gogle-drive",
    "config": [
        {
            "type": "googledrive",
            "parameters": {
                "settings_file": "evaluation_examples/settings/googledrive/settings.yml",
                "operation": [
                    "delete",
                    "mkdirs"
                ],
                "args": [
                    {
                        "query": "title = 'Meeting-Agenda.docx' or title = 'Meeting-Agenda.pdf'",
                        "trash": False
                    },
                    {
                        "path": [
                            "meetings"
                        ]
                    }
                ]
            }
        },
        {
            "type": "launch",
            "parameters": {
                "command": [
                    "google-chrome",
                    "--remote-debugging-port=1337"
                ]
            }
        },
        {
            "type": "launch",
            "parameters": {
                "command": [
                    "socat",
                    "tcp-listen:9222,fork",
                    "tcp:localhost:1337"
                ]
            }
        },
        {
            "type": "chrome_open_tabs",
            "parameters": {
                "urls_to_open": [
                    "https://news.google.com",
                    "https://x.com",
                    "https://www.deepl.com"
                ]
            }
        },
        {
            "type": "login",
            "parameters": {
                "settings_file": "evaluation_examples/settings/google/settings.json",
                "platform": "googledrive"
            }
        },
        {
            "type": "download",
            "parameters": {
                "files": [
                    {
                        "url": "https://drive.usercontent.google.com/download?id=1mIPe4P5b5WPoYI1zegSXjZ1Fhjjvr5Zk&export=download&authuser=0&confirm=t",
                        "path": "/home/user/Desktop/Meeting-Agenda.docx"
                    }
                ]
            }
        },
        {
            "type": "launch",
            "parameters": {
                "command": [
                    "libreoffice",
                    "--writer",
                    "/home/user/Desktop/Meeting-Agenda.docx"
                ]
            }
        }
    ],
    "trajectory": "trajectories/",
    "related_apps": [
        "libreoffice_writer",
        "chrome"
    ],
    "evaluator": {
        "func": "compare_pdfs",
        "result": {
            "type": "googledrive_file",
            "settings_file": "evaluation_examples/settings/googledrive/settings.yml",
            "path": [
                "meetings",
                "Meeting-Agenda.pdf"
            ],
            "dest": "Meeting-Agenda.pdf"
        },
        "expected": {
            "type": "cloud_file",
            "path": "https://drive.usercontent.google.com/download?id=1vY9GCxczkuom8bOU6SVHe9pTYgGIm1_M&export=download&authuser=0&confirm=t",
            "dest": "gold-Meeting-Agenda.pdf"
        }
    }
}



In [75]:
TYPE_TO_LOOK = ["googledrive", "login", "googledrive_file"]

In [76]:
example["config"][4]["parameters"]["platform"]

'googledrive'

In [77]:
for item in example["config"]:
    if item["type"] in TYPE_TO_LOOK:
        # setting_file = item["parameters"]["settings_file"]
        file_type = item["parameters"]["settings_file"].split(".")[-1]
        if file_type == "json":
            item["parameters"]["settings_file"] = "change json file"
        else:
            item["parameters"]["settings_file"] = "change yml file"

In [78]:
path = example["evaluator"]["result"]

if path["type"] in TYPE_TO_LOOK and path["settings_file"].split(".")[-1] == "yml":
    path["settings_file"] = "change yml file"

In [79]:
example["evaluator"]

{'func': 'compare_pdfs',
 'result': {'type': 'googledrive_file',
  'settings_file': 'change yml file',
  'path': ['meetings', 'Meeting-Agenda.pdf'],
  'dest': 'Meeting-Agenda.pdf'},
 'expected': {'type': 'cloud_file',
  'path': 'https://drive.usercontent.google.com/download?id=1vY9GCxczkuom8bOU6SVHe9pTYgGIm1_M&export=download&authuser=0&confirm=t',
  'dest': 'gold-Meeting-Agenda.pdf'}}

In [25]:
test = {
    "t1": "t11",
    "t2": "t22",
}

In [ ]:
windows, vscode, thunderbird, multi-apps

In [32]:
for example in test.keys():
    print(test[example])

t11
t22


In [24]:
print(example)

{'id': '22a4636f-8179-4357-8e87-d1743ece1f81', 'snapshot': 'chrome', 'instruction': 'Please help me convert the file "Meeting-Agenda.docx" to a pdf file and upload to the folder \'meetings\' in my google drive.', 'source': 'https://marketplace.uipath.com/listings/merge-pdfs-from-gmail-email-attachments-and-upload-to-gogle-drive', 'config': [{'type': 'googledrive', 'parameters': {'settings_file': 'change yml file', 'operation': ['delete', 'mkdirs'], 'args': [{'query': "title = 'Meeting-Agenda.docx' or title = 'Meeting-Agenda.pdf'", 'trash': False}, {'path': ['meetings']}]}}, {'type': 'launch', 'parameters': {'command': ['google-chrome', '--remote-debugging-port=1337']}}, {'type': 'launch', 'parameters': {'command': ['socat', 'tcp-listen:9222,fork', 'tcp:localhost:1337']}}, {'type': 'chrome_open_tabs', 'parameters': {'urls_to_open': ['https://news.google.com', 'https://x.com', 'https://www.deepl.com']}}, {'type': 'login', 'parameters': {'settings_file': 'change json file', 'platform': 'g

In [ ]:
# Case 1
env = OSWorld()
proc = OSWorldProcessor()

task = next(proc)
env.reset(task)

# Pros
# - one class that handles data ops
# - another for env
# - wanna use benchmark tasks? just use the processor
# - wanna process your custom slice of the benchmarks? just use the processor 
# - wanna use your custom task config? just dont use the processor

# Cons
# - additional class 

In [ ]:
# Case 2
env = OSWorld(OSWorldProcessor("evaluation_examples/"), secrets, use_benchmark=True)

env.reset()

# Pros
# - single class object to handle benchmark tasks or custom tasks

# Cons
# - intermingling between data ops and env ops
# - confuses the init for env; maybe a bit too overloading

In [ ]:
# Case 3

# OSWorld <- OSWorldAdapter <- OSWorldProcessor 
# OSWorldEnv -> OSWorld <- OSWorldProcessor

env = OSWorld()
proc = OSWorldProcessor("evaluation_examples/")

# Pros
# - scalable, modular, robust

# Cons
# - more effort
# - more maintenance
# - more complexity

In [ ]:
{
  "id": "0c825995-5b70-4526-b663-113f4c999dd2",
  "snapshot": "libreoffice_calc",
  "instruction": "I'm working on a comprehensive report for our environmental policy review meeting next week. I need to integrate key insights from an important document, which is a guidebook on the Green Economy, where I'm particularly interested in the 'Introduction' section. Could you extract this section and compile them into a new Google Doc named 'environment_policy_report (draft)' under /environment_policy folder? This will significantly aid in our discussion on aligning our environmental policies with sustainable and green economic practices. Thanks!",
  "source": "authors",
  "config": [
    {
      "type": "googledrive",
      "parameters": {
        "settings_file": "evaluation_examples/settings/googledrive/settings.yml",
        "operation": [
          "delete"
        ],
        "args": [
          {
            "query": "title = 'environment_policy_report (draft).doc' or title = 'environment_policy_report (draft).docx' or title = 'environment_policy_report (draft)'",
            "trash": false
          }
        ]
      }
    },
    {
      "type": "launch",
      "parameters": {
        "command": [
          "google-chrome",
          "--remote-debugging-port=1337"
        ]
      }
    },
    {
      "type": "launch",
      "parameters": {
        "command": [
          "socat",
          "tcp-listen:9222,fork",
          "tcp:localhost:1337"
        ]
      }
    },
    {
      "type": "login",
      "parameters": {
        "settings_file": "evaluation_examples/settings/google/settings.json",
        "platform": "googledrive"
      }
    },
    {
      "type": "command",
      "parameters": {
        "command": [
          "mkdir",
          "-p",
          "/home/user/Desktop/wwf"
        ]
      }
    },
    {
      "type": "download",
      "parameters": {
        "files": [
          {
            "path": "/home/user/Desktop/wwf/lpr_living_planet_report_2016.pdf",
            "url": "https://drive.google.com/uc?id=19NCdw_MVP6nH5nC6okYYe8U1mJABfTRK&export=download"
          },
          {
            "path": "/home/user/Desktop/wwf/279c656a32_ENGLISH_FULL.pdf",
            "url": "https://drive.google.com/uc?id=1ckH1NetfImQ9EyONTO-ZFWA8m8VIUFvD&export=download"
          },
          {
            "path": "/home/user/Desktop/wwf/7g37j96psg_WWF_AR2021_spreads.pdf",
            "url": "https://drive.google.com/uc?id=1cxLTzmqDKMomOyvho29lvFvhRnb0Y8__&export=download"
          },
          {
            "path": "/home/user/Desktop/GE Guidebook.pdf",
            "url": "https://drive.google.com/uc?id=1KzC_R3eI3Rmgwz5bkcI8Ohv7ebOrU-Is&export=download"
          },
          {
            "path": "/home/user/Desktop/assessing_and_reporting_water_quality(q&a).pdf",
            "url": "https://drive.google.com/uc?id=1LFojf3Weflv3fVdrZrgTY1iUaRdbT9kG&export=download"
          }
        ]
      }
    }
  ],
  "trajectory": "trajectories/0c825995-5b70-4526-b663-113f4c999dd2",
  "related_apps": [
    "libreoffice_calc",
    "chrome",
    "os"
  ],
  "evaluator": {
    "func": "compare_docx_files",
    "result": {
      "type": "googledrive_file",
      "settings_file": "evaluation_examples/settings/googledrive/settings.yml",
      "path": ["environment_policy", "environment_policy_report (draft)"],
      "dest": "environment_policy_report (draft).docx"
    },
    "expected": {
      "type": "cloud_file",
      "path": "https://drive.google.com/uc?id=1A2ti9JncAfIa6ks7FTJWHtYlZo-68FtM&export=download",
      "dest": "environment_policy_report (draft)_gold.docx"
    },
    "options": {
      "content_only": true
    }
  }
}


In [ ]:
env = DesktopEnv()
env.reset()

Googledrive
- settings.yml
  - client_secrets.json
  - credentials.json (write path)

Google
- settings.json

In [2]:
google_credentials_config = {
    "googledrive_settings": {  # settings.yml
        'client_config_backend': 'file',
        'client_config_file': "evaluation_examples/settings/googledrive/client_secrets.json",  # User created.
        'save_credentials': True,
        'save_credentials_backend': 'file',
        'save_credentials_file': "evaluation_examples/settings/googledrive/credentials.json",  # User specified.
        'get_refresh_token': True
    },
    "google_settings": {  # settings.json
        "email": "vincentsimulartest@gmail.com",
        "password": ""
    }
}

# settings.yml (googledrive)
    # client_secrets.json
    # credentials.json path specified by user
# settings.json (google)

In [3]:
google_credentials_config['googledrive_settings']

{'client_config_backend': 'file',
 'client_config_file': 'evaluation_examples/settings/googledrive/client_secrets.json',
 'save_credentials': True,
 'save_credentials_backend': 'file',
 'save_credentials_file': 'evaluation_examples/settings/googledrive/credentials.json',
 'get_refresh_token': True}

In [4]:
google_credentials_config['google_settings']

{'email': 'vincentsimulartest@gmail.com', 'password': ''}

In [ ]:
read and write

## Testing Implementation

In [1]:
from agential.benchmarks.computer_use.osworld.osworld_benchmark import OSWorldBenchmark
from agential.benchmarks.computer_use.osworld.osworld_mediator import OSWorldMediator

/Users/chuongnguyen/opt/anaconda3/envs/agential/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/chuongnguyen/opt/anaconda3/envs/agential/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <910128BE-E745-359A-8A99-049B245C2D92> /Users/chuongnguyen/opt/anaconda3/envs/agential/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <23AEC8EA-0BA7-322C-93F3-4D2D1EA66F06> /Users/chuongnguyen/opt/anaconda3/envs/agential/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/chuongnguyen/opt/anaconda3/envs/agential/lib/python3.11/site-packages/tqdm/auto.py:21: Tqd

In [2]:
UBUNTUO_VMX = "code_storage/vmware_vm_data/Ubuntu0/Ubuntu0.vmx"

In [3]:
import os
import subprocess

# Add vmrun path to the PATH environment variable
os.environ["PATH"] += ":/Applications/VMware Fusion.app/Contents/Public"

# Verify the new PATH
print(os.environ['PATH'])

# Run vmrun using the full path
command = ["/Applications/VMware Fusion.app/Contents/Public/vmrun", "your_command_here"]
result = subprocess.run(command, capture_output=True, text=True)

# Print output or error
print(result.stdout)
print(result.stderr)


/Users/chuongnguyen/opt/anaconda3/envs/agential/bin:/Users/chuongnguyen/.local/pipx/venvs/poetry/bin:/Users/chuongnguyen/.local/pipx/venvs/poetry/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Applications/VMware Fusion.app/Contents/Public
Error: Unrecognized command: your_command_here




In [4]:
examples_dir: str = "agential/benchmarks/computer_use/osworld/evaluation_examples/examples"
path_to_google_settings: str = "settings/google/settings.json"
path_to_googledrive_settings: str = "settings/googledrive/settings.yml"
osworld_benchmark: OSWorldBenchmark = OSWorldBenchmark(action_space="pyautogui", path_to_vm=UBUNTUO_VMX)

Starting VMware VM...


In [5]:
osworld = OSWorldMediator(
    examples_dir=examples_dir,
    path_to_google_settings=path_to_google_settings,
    path_to_googledrive_settings=path_to_googledrive_settings,
    osworld_benchmark=osworld_benchmark
)

In [6]:
domain = "multi_apps"
task_id = "22a4636f-8179-4357-8e87-d1743ece1f81"


In [7]:
osworld.data[domain][task_id]

{'id': '22a4636f-8179-4357-8e87-d1743ece1f81',
 'snapshot': 'chrome',
 'instruction': 'Please help me convert the file "Meeting-Agenda.docx" to a pdf file and upload to the folder \'meetings\' in my google drive.',
 'source': 'https://marketplace.uipath.com/listings/merge-pdfs-from-gmail-email-attachments-and-upload-to-gogle-drive',
 'config': [{'type': 'googledrive',
   'parameters': {'settings_file': 'settings/googledrive/settings.yml',
    'operation': ['delete', 'mkdirs'],
    'args': [{'query': "title = 'Meeting-Agenda.docx' or title = 'Meeting-Agenda.pdf'",
      'trash': False},
     {'path': ['meetings']}]}},
  {'type': 'launch',
   'parameters': {'command': ['google-chrome',
     '--remote-debugging-port=1337']}},
  {'type': 'launch',
   'parameters': {'command': ['socat',
     'tcp-listen:9222,fork',
     'tcp:localhost:1337']}},
  {'type': 'chrome_open_tabs',
   'parameters': {'urls_to_open': ['https://news.google.com',
     'https://x.com',
     'https://www.deepl.com']}},


In [8]:
update = osworld.update_credential(
    domain=domain,
    task_id=task_id
)

In [9]:
print(os.path.exists(path_to_google_settings))

True


In [7]:
obs = osworld.reset(
    domain=domain
)

Starting VMware VM...


An error occurred while trying to send the request: HTTPConnectionPool(host='172.16.190.136', port=5000): Max retries exceeded with url: /setup/launch (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x15db4f550>: Failed to establish a new connection: [Errno 65] No route to host'))
An error occurred while trying to send the request: HTTPConnectionPool(host='172.16.190.136', port=5000): Max retries exceeded with url: /setup/launch (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x15db5ed50>: Failed to establish a new connection: [Errno 65] No route to host'))


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [17]:
print(str.upper("osworld_processor"))

OSWORLD_PROCESSOR


In [9]:
EXAMPLES_DIR: str = (
    "agential/benchmarks/computer_use/osworld/evaluation_examples/examples"
)
PATH_TO_GOOGLE_SETTINGS: str = (
    "agential/benchmarks/computer_use/osworld/evaluation_examples/settings/google/settings.json"
)
PATH_TO_GOOGLEDRIVE_SETTINGS: str = (
    "agential/benchmarks/computer_use/osworld/evaluation_examples/settings/googledrive/settings.yml"
)
DOMAIN = "multi_apps"
TASK_ID = "0c825995-5b70-4526-b663-113f4c999dd2"

In [10]:
example_1_input = {
    "id": "4e9f0faf-2ecc-4ae8-a804-28c9a75d1ddc",
    "snapshot": "chrome",
    "instruction": "Could you help me extract data in the table from a new invoice uploaded to my Google Drive, then export it to a Libreoffice calc .xlsx file in the desktop?",
    "source": "https://marketplace.uipath.com/listings/extract-data-from-a-new-invoice-file-in-google-drive-and-store-it-in-google-sheets4473",
    "config": [
        {
            "type": "googledrive",
            "parameters": {
                "settings_file": "evaluation_examples/settings/googledrive/settings.yml",
                "operation": ["delete", "upload"],
                "args": [
                    {"query": "title = 'invoice.pdf'", "trash": False},
                    {
                        "url": "https://drive.usercontent.google.com/download?id=1KAhoPFM0AU2dgn_NRt3y7CjOr9Er4vwD&export=download&authuser=0&confirm=t&uuid=e8528cd1-5106-45f3-a644-e1bbf5e08278&at=APZUnTUnTuXfV2Ted_9Wv2QomMvA:1706181110208",
                        "path": ["invoice.pdf"],
                    },
                ],
            },
        },
        {
            "type": "launch",
            "parameters": {
                "command": ["google-chrome", "--remote-debugging-port=1337"]
            },
        },
        {
            "type": "launch",
            "parameters": {
                "command": ["socat", "tcp-listen:9222,fork", "tcp:localhost:1337"]
            },
        },
        {
            "type": "chrome_open_tabs",
            "parameters": {
                "urls_to_open": ["https://news.google.com", "https://x.com"]
            },
        },
        {
            "type": "login",
            "parameters": {
                "settings_file": "evaluation_examples/settings/google/settings.json",
                "platform": "googledrive",
            },
        },
    ],
    "trajectory": "trajectories/",
    "related_apps": ["libreoffice_calc", "chrome"],
    "evaluator": {
        "func": "compare_table",
        "result": {
            "type": "vm_file",
            "path": "/home/user/Desktop/invoice.xlsx",
            "dest": "invoice.xlsx",
        },
        "expected": {
            "type": "cloud_file",
            "path": "https://drive.usercontent.google.com/download?id=1gkATnr8bk4JKQbzXZvzifoAQUA2sx5da&export=download&authuser=0&confirm=t&uuid=64ed0549-1627-49e8-8228-1e1925d6f6f7&at=APZUnTXkCm24SrOPuO5C6v4M3BiB:1706181091638",
            "dest": "invoice_gold.xlsx",
        },
        "options": {
            "rules": [
                {"type": "sheet_data", "sheet_idx0": "RI0", "sheet_idx1": "EI0"}
            ]
        },
    },
}
example_2_input = {
    "id": "0c825995-5b70-4526-b663-113f4c999dd2",
    "snapshot": "libreoffice_calc",
    "instruction": "I'm working on a comprehensive report for our environmental policy review meeting next week. I need to integrate key insights from an important document, which is a guidebook on the Green Economy, where I'm particularly interested in the 'Introduction' section. Could you extract this section and compile them into a new Google Doc named 'environment_policy_report (draft)' under /environment_policy folder? This will significantly aid in our discussion on aligning our environmental policies with sustainable and green economic practices. Thanks!",
    "source": "authors",
    "config": [
        {
            "type": "googledrive",
            "parameters": {
                "settings_file": "evaluation_examples/settings/googledrive/settings.yml",
                "operation": ["delete"],
                "args": [
                    {
                        "query": "title = 'environment_policy_report (draft).doc' or title = 'environment_policy_report (draft).docx' or title = 'environment_policy_report (draft)'",
                        "trash": False,
                    }
                ],
            },
        },
        {
            "type": "launch",
            "parameters": {
                "command": ["google-chrome", "--remote-debugging-port=1337"]
            },
        },
        {
            "type": "launch",
            "parameters": {
                "command": ["socat", "tcp-listen:9222,fork", "tcp:localhost:1337"]
            },
        },
        {
            "type": "login",
            "parameters": {
                "settings_file": "evaluation_examples/settings/google/settings.json",
                "platform": "googledrive",
            },
        },
        {
            "type": "command",
            "parameters": {"command": ["mkdir", "-p", "/home/user/Desktop/wwf"]},
        },
        {
            "type": "download",
            "parameters": {
                "files": [
                    {
                        "path": "/home/user/Desktop/wwf/lpr_living_planet_report_2016.pdf",
                        "url": "https://drive.google.com/uc?id=19NCdw_MVP6nH5nC6okYYe8U1mJABfTRK&export=download",
                    },
                    {
                        "path": "/home/user/Desktop/wwf/279c656a32_ENGLISH_FULL.pdf",
                        "url": "https://drive.google.com/uc?id=1ckH1NetfImQ9EyONTO-ZFWA8m8VIUFvD&export=download",
                    },
                    {
                        "path": "/home/user/Desktop/wwf/7g37j96psg_WWF_AR2021_spreads.pdf",
                        "url": "https://drive.google.com/uc?id=1cxLTzmqDKMomOyvho29lvFvhRnb0Y8__&export=download",
                    },
                    {
                        "path": "/home/user/Desktop/GE Guidebook.pdf",
                        "url": "https://drive.google.com/uc?id=1KzC_R3eI3Rmgwz5bkcI8Ohv7ebOrU-Is&export=download",
                    },
                    {
                        "path": "/home/user/Desktop/assessing_and_reporting_water_quality(q&a).pdf",
                        "url": "https://drive.google.com/uc?id=1LFojf3Weflv3fVdrZrgTY1iUaRdbT9kG&export=download",
                    },
                ]
            },
        },
    ],
    "trajectory": "trajectories/0c825995-5b70-4526-b663-113f4c999dd2",
    "related_apps": ["libreoffice_calc", "chrome", "os"],
    "evaluator": {
        "func": "compare_docx_files",
        "result": {
            "type": "googledrive_file",
            "settings_file": "evaluation_examples/settings/googledrive/settings.yml",
            "path": ["environment_policy", "environment_policy_report (draft)"],
            "dest": "environment_policy_report (draft).docx",
        },
        "expected": {
            "type": "cloud_file",
            "path": "https://drive.google.com/uc?id=1A2ti9JncAfIa6ks7FTJWHtYlZo-68FtM&export=download",
            "dest": "environment_policy_report (draft)_gold.docx",
        },
        "options": {"content_only": True},
    },
}
temp_data_input = {
    "multi_apps": {
        "4e9f0faf-2ecc-4ae8-a804-28c9a75d1ddc": example_1_input,
        "0c825995-5b70-4526-b663-113f4c999dd2": example_2_input,
    }
}

example_1_output = {
    "id": "4e9f0faf-2ecc-4ae8-a804-28c9a75d1ddc",
    "snapshot": "chrome",
    "instruction": "Could you help me extract data in the table from a new invoice uploaded to my Google Drive, then export it to a Libreoffice calc .xlsx file in the desktop?",
    "source": "https://marketplace.uipath.com/listings/extract-data-from-a-new-invoice-file-in-google-drive-and-store-it-in-google-sheets4473",
    "config": [
        {
            "type": "googledrive",
            "parameters": {
                "settings_file": PATH_TO_GOOGLEDRIVE_SETTINGS,
                "operation": ["delete", "upload"],
                "args": [
                    {"query": "title = 'invoice.pdf'", "trash": False},
                    {
                        "url": "https://drive.usercontent.google.com/download?id=1KAhoPFM0AU2dgn_NRt3y7CjOr9Er4vwD&export=download&authuser=0&confirm=t&uuid=e8528cd1-5106-45f3-a644-e1bbf5e08278&at=APZUnTUnTuXfV2Ted_9Wv2QomMvA:1706181110208",
                        "path": ["invoice.pdf"],
                    },
                ],
            },
        },
        {
            "type": "launch",
            "parameters": {
                "command": ["google-chrome", "--remote-debugging-port=1337"]
            },
        },
        {
            "type": "launch",
            "parameters": {
                "command": ["socat", "tcp-listen:9222,fork", "tcp:localhost:1337"]
            },
        },
        {
            "type": "chrome_open_tabs",
            "parameters": {
                "urls_to_open": ["https://news.google.com", "https://x.com"]
            },
        },
        {
            "type": "login",
            "parameters": {
                "settings_file": PATH_TO_GOOGLE_SETTINGS,
                "platform": "googledrive",
            },
        },
    ],
    "trajectory": "trajectories/",
    "related_apps": ["libreoffice_calc", "chrome"],
    "evaluator": {
        "func": "compare_table",
        "result": {
            "type": "vm_file",
            "path": "/home/user/Desktop/invoice.xlsx",
            "dest": "invoice.xlsx",
        },
        "expected": {
            "type": "cloud_file",
            "path": "https://drive.usercontent.google.com/download?id=1gkATnr8bk4JKQbzXZvzifoAQUA2sx5da&export=download&authuser=0&confirm=t&uuid=64ed0549-1627-49e8-8228-1e1925d6f6f7&at=APZUnTXkCm24SrOPuO5C6v4M3BiB:1706181091638",
            "dest": "invoice_gold.xlsx",
        },
        "options": {
            "rules": [
                {"type": "sheet_data", "sheet_idx0": "RI0", "sheet_idx1": "EI0"}
            ]
        },
    },
}
example_2_output = {
    "id": "0c825995-5b70-4526-b663-113f4c999dd2",
    "snapshot": "libreoffice_calc",
    "instruction": "I'm working on a comprehensive report for our environmental policy review meeting next week. I need to integrate key insights from an important document, which is a guidebook on the Green Economy, where I'm particularly interested in the 'Introduction' section. Could you extract this section and compile them into a new Google Doc named 'environment_policy_report (draft)' under /environment_policy folder? This will significantly aid in our discussion on aligning our environmental policies with sustainable and green economic practices. Thanks!",
    "source": "authors",
    "config": [
        {
            "type": "googledrive",
            "parameters": {
                "settings_file": PATH_TO_GOOGLEDRIVE_SETTINGS,
                "operation": ["delete"],
                "args": [
                    {
                        "query": "title = 'environment_policy_report (draft).doc' or title = 'environment_policy_report (draft).docx' or title = 'environment_policy_report (draft)'",
                        "trash": False,
                    }
                ],
            },
        },
        {
            "type": "launch",
            "parameters": {
                "command": ["google-chrome", "--remote-debugging-port=1337"]
            },
        },
        {
            "type": "launch",
            "parameters": {
                "command": ["socat", "tcp-listen:9222,fork", "tcp:localhost:1337"]
            },
        },
        {
            "type": "login",
            "parameters": {
                "settings_file": PATH_TO_GOOGLE_SETTINGS,
                "platform": "googledrive",
            },
        },
        {
            "type": "command",
            "parameters": {"command": ["mkdir", "-p", "/home/user/Desktop/wwf"]},
        },
        {
            "type": "download",
            "parameters": {
                "files": [
                    {
                        "path": "/home/user/Desktop/wwf/lpr_living_planet_report_2016.pdf",
                        "url": "https://drive.google.com/uc?id=19NCdw_MVP6nH5nC6okYYe8U1mJABfTRK&export=download",
                    },
                    {
                        "path": "/home/user/Desktop/wwf/279c656a32_ENGLISH_FULL.pdf",
                        "url": "https://drive.google.com/uc?id=1ckH1NetfImQ9EyONTO-ZFWA8m8VIUFvD&export=download",
                    },
                    {
                        "path": "/home/user/Desktop/wwf/7g37j96psg_WWF_AR2021_spreads.pdf",
                        "url": "https://drive.google.com/uc?id=1cxLTzmqDKMomOyvho29lvFvhRnb0Y8__&export=download",
                    },
                    {
                        "path": "/home/user/Desktop/GE Guidebook.pdf",
                        "url": "https://drive.google.com/uc?id=1KzC_R3eI3Rmgwz5bkcI8Ohv7ebOrU-Is&export=download",
                    },
                    {
                        "path": "/home/user/Desktop/assessing_and_reporting_water_quality(q&a).pdf",
                        "url": "https://drive.google.com/uc?id=1LFojf3Weflv3fVdrZrgTY1iUaRdbT9kG&export=download",
                    },
                ]
            },
        },
    ],
    "trajectory": "trajectories/0c825995-5b70-4526-b663-113f4c999dd2",
    "related_apps": ["libreoffice_calc", "chrome", "os"],
    "evaluator": {
        "func": "compare_docx_files",
        "result": {
            "type": "googledrive_file",
            "settings_file": PATH_TO_GOOGLEDRIVE_SETTINGS,
            "path": ["environment_policy", "environment_policy_report (draft)"],
            "dest": "environment_policy_report (draft).docx",
        },
        "expected": {
            "type": "cloud_file",
            "path": "https://drive.google.com/uc?id=1A2ti9JncAfIa6ks7FTJWHtYlZo-68FtM&export=download",
            "dest": "environment_policy_report (draft)_gold.docx",
        },
        "options": {"content_only": True},
    },
}
temp_data_output = {
    "multi_apps": {
        "4e9f0faf-2ecc-4ae8-a804-28c9a75d1ddc": example_1_output,
        "0c825995-5b70-4526-b663-113f4c999dd2": example_2_output,
    }
}

In [14]:
from typing import Dict, Any
TYPE_TO_LOOK = ["googledrive", "login", "googledrive_file"]
path_to_googledrive_settings = PATH_TO_GOOGLEDRIVE_SETTINGS
path_to_google_settings = PATH_TO_GOOGLE_SETTINGS

def _change_credential(example: Dict[str, Any]) -> Any:
        """Modifies credential settings in a given example based on file type.

        Args:
            example (Dict[str, Any]): The task configuration to be updated.

        Returns:
            Dict[str, Any]: The updated task configuration.
        """
        print(example["config"])
        for item in example["config"]:
            if item["type"] in TYPE_TO_LOOK:
                file_type = item["parameters"]["settings_file"].split(".")[-1]
                if file_type == "yml":
                    item["parameters"][
                        "settings_file"
                    ] = path_to_googledrive_settings
                else:
                    item["parameters"]["settings_file"] = path_to_google_settings

        path = example["evaluator"]["result"]
        if (
            path["type"] in TYPE_TO_LOOK
            and path["settings_file"].split(".")[-1] == "yml"
        ):
            path["settings_file"] = path_to_googledrive_settings

        return example


In [15]:
data = temp_data_input

In [23]:
print(data['multi_apps'].keys())

dict_keys(['4e9f0faf-2ecc-4ae8-a804-28c9a75d1ddc', '0c825995-5b70-4526-b663-113f4c999dd2'])


In [25]:
for each_domain in data.keys():
    for each_task in data[each_domain].keys():
        data[each_domain][each_task] = _change_credential(
            data[each_domain][each_task]
        )

[{'type': 'googledrive', 'parameters': {'settings_file': 'evaluation_examples/settings/googledrive/settings.yml', 'operation': ['delete', 'upload'], 'args': [{'query': "title = 'invoice.pdf'", 'trash': False}, {'url': 'https://drive.usercontent.google.com/download?id=1KAhoPFM0AU2dgn_NRt3y7CjOr9Er4vwD&export=download&authuser=0&confirm=t&uuid=e8528cd1-5106-45f3-a644-e1bbf5e08278&at=APZUnTUnTuXfV2Ted_9Wv2QomMvA:1706181110208', 'path': ['invoice.pdf']}]}}, {'type': 'launch', 'parameters': {'command': ['google-chrome', '--remote-debugging-port=1337']}}, {'type': 'launch', 'parameters': {'command': ['socat', 'tcp-listen:9222,fork', 'tcp:localhost:1337']}}, {'type': 'chrome_open_tabs', 'parameters': {'urls_to_open': ['https://news.google.com', 'https://x.com']}}, {'type': 'login', 'parameters': {'settings_file': 'evaluation_examples/settings/google/settings.json', 'platform': 'googledrive'}}]
[{'type': 'googledrive', 'parameters': {'settings_file': 'evaluation_examples/settings/googledrive/s